# Homework 3: Twitter POS tagging

Student Name: Zihang Su

Student ID: 710118

Python version used: python 3

## General info

<b>Due date</b>: 11pm, Sunday April 15

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be adapting a POS tagger to Twitter data, starting from a tagger trained on Penn Treebank. You will also use prior information on the Twitter tagset to obtain better performance. Finally, you will also analyse your results in a more fine-grained way. For extra credits, you will implement the Expectation-Maximisation algorithm.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


### Part 1: Preprocessing (2.0)

<b>Instructions</b>: you first task is to preprocess the data. We will use two datasets for training: 1) the Penn Treebank sample used in the workshops and 2) the Twitter samples data you used in Homework 1. In order to adapt the tagger to the Twitter data we need to built a *joint* vocabulary containing all the word types in PTB and the twitter_samples corpora. So, in addition to preprocessing, your code should also build this vocabulary. Finally, you should also store the tagset, for reasons that will be clearer later.

<b>Important</b>: you are allowed to reuse all the code from the workshop notebooks. In fact you are encouraged to do so as much of this homework will be based on these notebooks.

The vocabulary and the tagset should be stored in Python dictionaries, mapping each word (or tag) to an index (integer). This is similar to what is done in the W6/W7 workshop notebooks. The preprocessed corpora should contain indices only, as in the workshop.

Let's start with the PTB data. You should iterate over all sentences and words, and build the vocabulary and the tagset. Important: make sure you <b>lowercase</b> words before they are added to the dictionary. You should also generate the preprocessed corpus. It should be a list where each element is a tagged sentence, represented as another list of (word, tag) indices (which should correspond to the original words/tags). Print the first preprocessed sentence, the index for the word 'electricity' and the length of the full tagset. (0.5)


In [1]:
# notice: I re-use the code from W6 and W7

from nltk.corpus import treebank
corpus = treebank.tagged_sents()

word_numbers = {}
tag_numbers = {}

num_corpus = []
for sent in corpus:
    num_sent = []
    for word, tag in sent:
        wi = word_numbers.setdefault(word.lower(), len(word_numbers))
        ti = tag_numbers.setdefault(tag, len(tag_numbers))
        num_sent.append((wi, ti))
    num_corpus.append(num_sent)

print(num_corpus[0])
print(word_numbers['electricity'])
print(len(tag_numbers))

[(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (2, 1), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 7), (12, 4), (13, 8), (14, 0), (15, 2), (16, 10)]
1095
46


<b>Instructions</b>: now you should do the same with the twitter_samples dataset. From now on, we will refer this dataset as the **training** tweets. Since this data is not tagged, the preprocessed corpus should be a list where each element is another list containing indices only (instead of (word, tag) tuples). A tokenised version of twitter_samples is available through the method .tokenized(), use this method to read your corpus. Besides generating the corpus, you should also **update** the vocabulary with the new words from this corpus.

There are two things to keep in mind when doing this process:

1) We will perform a bit more of preprocessing in this dataset, besides lowercasing. Specifically, you should replace special tokens with special symbols, as follows:
- Username mentions are tokens that start with '@': replace these tokens with 'USER_TOKEN'
- Hashtags are tokens that start with '#': replace these with 'HASHTAG_TOKEN'
- Retweets are represented as the token 'RT' (or 'rt' if you lowercase first): replace these with 'RETWEET_TOKEN'
- URLs are tokens that start with 'https://' or 'http://': replace these with 'URL_TOKEN'

2) **Do not create a new vocabulary**. Instead, you should update the vocabulary built from PTB with any new words present in this corpus. These should *include* the special tokens defined above but *not* the original un-preprocessed tokens.

The easiest way to do these steps is by doing 3 passes over the data: preprocess the words first, update the vocabulary and finally convert the corpus into the list format described above. However, it is possible to do all of this in one pass only.

Print the first sentence from your preprocessed corpora, the index for the word 'electricity' and the index for 'HASHTAG_TOKEN'. (0.5)

In [2]:
import re
from nltk.corpus import twitter_samples
tweets_corpus = twitter_samples.tokenized()

encoded_tweets = []
for sent in tweets_corpus:
    encode = []
    for word in sent:
        word = word.lower()
        if re.match(r'^@(.*)$', word):
            word = 'USER_TOKEN'
        if re.match(r'^#(.*)$', word):
            word = 'HASHTAG_TOKEN'
        if re.match(r'^rt$', word):
            word = 'RETWEET_TOKEN'
        if re.match(r'^https://(.*)$', word) or re.match(r'^http://(.*)$', word):
            word = 'URL_TOKEN'
        
        # get the index of word if word in dict
        # if word not in dict, automatically assign one more new item
        word_index = word_numbers.setdefault(word, len(word_numbers))
        encode.append(word_index)

    encoded_tweets.append(encode)
    
print(encoded_tweets[0])
print(word_numbers['electricity'])
print(word_numbers['HASHTAG_TOKEN'])

[11387, 182, 11388, 11389]
1095
11409


<b>Instructions:</b> now we will preprocess the tagged twitter corpus used in W7 (Ritter et al.). This dataset will be referred from now on as **test** tweets. Before you do that though, you should update the tagset.

You might have noticed this in the workshop but this dataset has a few extra tags, besides the PTB ones. These were added to incorporate specific phenomena that happens on Twitter:
- "USR": username mentions
- "HT": hashtags
- "RT": retweets
- "URL": URL addresses

Notice that these special tags correspond to the special tokens we preprocessed before. These steps will be important in Part 3 later.

There a few additional tags which are not specific to Twitter but are not present in the PTB sample:
- "VPP"
- "TD"
- "O"

You should add these new seven tags to the tagset you built when reading the PTB corpus.

Another task is to add an extra type to the vocabulary: `<unk>`. This is in order to account for unknown or out-of-vocabulary words.

Finally, build two "inverted indices" for the vocabulary and the tagset. These should be lists, where the "i"-th element should contain the word (or tag) corresponding to the index "i" in the vocabulary (or tagset).

After doing these tasks, print the index for `<unk>` and the length of your resulting tagset. (0.5)

In [3]:
tag_numbers.setdefault('USR', len(tag_numbers))
tag_numbers.setdefault('HT', len(tag_numbers))
tag_numbers.setdefault('RT', len(tag_numbers))
tag_numbers.setdefault('URL', len(tag_numbers))
tag_numbers.setdefault('VPP', len(tag_numbers))
tag_numbers.setdefault('TD', len(tag_numbers))
tag_numbers.setdefault('O', len(tag_numbers))
word_numbers.setdefault('<unk>', len(word_numbers))

word_names = [None] * len(word_numbers)
for word, index in word_numbers.items():
    word_names[index] = word
tag_names = [None] * len(tag_numbers)
for tag, index in tag_numbers.items():
    tag_names[index] = tag
    
print(word_numbers['<unk>'])
print(len(tag_names))

26069
53


<b>Instructions</b>: now we can read the test tweets. Store them in the same format as the PTB corpora (list of lists containing (word, tag) index tuples). Do the same preprocessing steps that you did for the training tweets (lowercasing + replace special tokens). However, **do not** update the vocabulary. Why? Because the test set should simulate a real-world scenario, where out-of-vocabulary words can appear. Instead, after preprocessing each word, you should check if that word is in the vocabulary. If yes, just replace it with its index, otherwise you should replace it with the index for the `<unk>` token. Remember: you can reuse the code from the workshop for this task. Just be mindful that in the workshop we stored words and tags in two separate lists: here you should have a single list, as in the PTB corpus you preprocessed above.

When reading the POS tags for the test tweets you should do some additional preprocessing. There are three tags in this dataset which correspond to PTB tags but are represented with different names:
- "(". In PTB, this is represented as "-LRB-"
- ")". In PTB, this is represented as "-RRB-"
- "NONE". In PTB, this is represented as "-NONE-"

As you build the corpus for the test tweets, you should check if the tag for a word is one of the above. If yes, you should use the PTB equivalent instead. In practice, it is sufficient to ensure you use the correct index for the corresponding tag, using your tagset dictionary. This concept is sometimes referred as *tag harmonisation*, where two different tagsets are mapped to each other.

After this, print the first sentence of your preprocessed corpus. (0.5)

In [4]:
import urllib
try:
    urllib.request.urlretrieve("https://github.com/aritter/twitter_nlp/raw/master/data/annotated/pos.txt","pos.txt")
except: # Python 2
    urllib.urlretrieve("https://github.com/aritter/twitter_nlp/raw/master/data/annotated/pos.txt","pos.txt")

test_tweets = []
with open('pos.txt') as f:
    sent = []
    for line in f:
        
        # .strip() removes all whitespace at the start and end, including spaces, tabs, newlines and carriage returns
        
        if line.strip() == '':
            test_tweets.append(sent)
            sent = []
        else:
            word, pos = line.strip().split()
            # preprocess the word
            word = word.lower()
            if re.match(r'^@(.*)$', word):
                word = 'USER_TOKEN'
            if re.match(r'^#(.*)$', word):
                word = 'HASHTAG_TOKEN'
            if re.match(r'^rt$', word):
                word = 'RETWEET_TOKEN'
            if re.match(r'^https://(.*)$', word) or re.match(r'^http://(.*)$', word):
                word = 'URL_TOKEN'
            
            # preprocess the tag
            if pos == '(':
                pos = '-LRB-'
            if pos == ')':
                pos = '-RRB-'
            if pos == 'NONE':
                pos = '-NONE-'
            
            # encode word and tag
            if word in word_names:
                word = word_numbers[word]
            else:
                word = word_numbers['<unk>']   
            pos = tag_numbers[pos]
            
            # append encoded tuple of (word, tag) into sentence
            sent.append((word, pos))
    
print(test_tweets[0])

[(11392, 46), (61, 19), (114, 11), (8, 7), (3224, 8), (170, 9), (325, 33), (1325, 19), (2375, 22), (3205, 12), (182, 9), (799, 2), (1522, 3), (16, 10), (8490, 0), (1146, 0), (2495, 0), (14039, 43), (26069, 0), (16, 10), (4263, 17), (1760, 4), (9464, 8), (2259, 17), (888, 4), (741, 8), (16, 10)]


<b>Hint</b>: if you did these steps correctly you should have 53 tags in your tagset and around 26000 words in your vocabulary.

### Part 2: Running the PTB tagger on the test tweets (1.5)

<b>Instructions</b>: your next task is to train a POS tagger on the PTB data and try it on the test tweets. This is exactly what we did in W7: feel free to reuse code. However, we are also gonna modify the code a bit.

Your first task is encapsulate the HMM training code into a function. You should name your function `count`. This function should take these input parameters:
- A tagged corpus, in the format described above (list of lists containing (word, tag) index tuples).
- The vocabulary (a dict).
- The tagset (a dict).

Output return values should contain:
- The initial tag probabilities (a vector).
- The transition probabilities (a matrix).
- The emission probabilities (a matrix).

Notice that in the workshop code the vocabulary and tagset were built as part of the training process. Here you should pass them explicitly as parameters instead. This is to ensure our tagger can take into account the words in the training tweets and the extra tags. Important: the workshop code initialise the probabilities with an `eps` value, to ensure you end up with non-zero probabilities for unseen events. You should do the same here.

After writing your function, run it on the PTB corpus to obtain the initial, transition and emission probabilities. (0.5)

In [5]:
import numpy as np

def count(tagged_corpus, vocabulary, tagset):
    S = len(tagset)
    V = len(vocabulary)

    # initalise
    eps = 0.1
    pi = eps * np.ones(S)
    A = eps * np.ones((S, S))
    O = eps * np.ones((S, V))

    # count
    for sent in tagged_corpus:
        last_tag = None
        for word, tag in sent:
            O[tag, word] += 1
            if last_tag == None:
                pi[tag] += 1
            else:
                A[last_tag, tag] += 1
            last_tag = tag
        
    # normalise
    pi /= np.sum(pi)
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
        
    return pi, A, O

initial, transition, emission = count(num_corpus, word_numbers, tag_numbers)

<b>Instructions</b>: now you should write a function for Viterbi. The input parameters are the same as in the workshop:
- The parameters (probabilities) of your HMM (a tuple (initial, transition, emission)).
- The input words (a list with numbers).

The output is slightly different though:
- A list of (word, tag) indices, containing the original input word and the predicted tag.

Run Viterbi on the test tweets and store the predictions in a list (might take a few seconds). Remember that in the processing part you stored the test tweets as (word, tag) indices lists: make sure your input to Viterbi are word index lists only. Print the first sentence of your predicted list. (0.5)

In [6]:
def viterbi(params, observations):
    pi, A, O = params
    M = len(observations)   # get number of input words
    S = pi.shape[0]   # get number of tags
    
    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf')
    backpointers = np.zeros((M, S), 'int')
    
    # base case
    alpha[0, :] = pi * O[:,observations[0]]      # transmition X emission
    
    # recursive case
    for t in range(1, M):
        for s2 in range(S):
            for s1 in range(S):
                score = alpha[t-1, s1] * A[s1, s2] * O[s2, observations[t]]
                if score > alpha[t, s2]:
                    alpha[t, s2] = score
                    backpointers[t, s2] = s1
    
    # now follow backpointers to resolve the state sequence
    reversed_pred_tags = []
    # return the last word index and the tag index of the largest score (word_index, tag_index)
    reversed_pred_tags.append((observations[-1], np.argmax(alpha[M-1,:]))) 
    for i in range(M-1, 0, -1):
        reversed_pred_tags.append((observations[i-1], backpointers[i, reversed_pred_tags[-1][1]]))
        
    return list(reversed(reversed_pred_tags))

# the test tweets:
word_index_test_tweets = []
for tweet in test_tweets:
    word_index = []
    for word, tag in tweet:
        word_index.append(word)
    word_index_test_tweets.append(word_index)

# run viterbi on test tweets:
pred_tags = []
for tweet in word_index_test_tweets:
    pred_tags.append(viterbi((initial, transition, emission), tweet))
print(pred_tags[0])

[(11392, 27), (61, 19), (114, 11), (8, 7), (3224, 8), (170, 9), (325, 33), (1325, 19), (2375, 22), (3205, 12), (182, 9), (799, 2), (1522, 3), (16, 10), (8490, 29), (1146, 8), (2495, 8), (14039, 10), (26069, 38), (16, 10), (4263, 29), (1760, 4), (9464, 8), (2259, 17), (888, 4), (741, 8), (16, 10)]


<b>Instructions</b>: you should now evaluate the results. Write a function that takes (word, tag) lists as inputs and outputs the tag sequence using the original tags in the tagset. Your inputs should be a sentence and the tag inverted index you built before.

Run this function on the predictions you obtained above **and** the test tweets, storing them in two separate lists. Finally, flat your predictions into a single list and do the same for the test tweets and report accuracy. (0.5)

In [7]:
from sklearn.metrics import accuracy_score

def tag_sequence(encoded_tagged_sentence):
    tags = []
    for tup in encoded_tagged_sentence:
        tag_index = tup[1]
        tags.append(tag_names[tag_index])
    return tags

tags_of_predictions = []
tags_of_test_tweets = []
length = len(pred_tags)
for i in range(length):
    tags_of_predictions += tag_sequence(pred_tags[i])
    tags_of_test_tweets += tag_sequence(test_tweets[i])

print(accuracy_score(tags_of_test_tweets, tags_of_predictions))

0.637141916365


### Part 3: Adapting the tagger using prior information (1.5)

<b>Instructions</b>: now your task is to adapt the tagger using prior information. What do we mean by that? Remember from part 1 that the twitter tagset has some extra tags, related to special tokens such as mentions and hashtags. In other words, **we know beforehand** that these special tokens **should** have these tags. However, because these tags never appear in the PTB data, the tagger has no such information. We are going to add this in order to improve the tagger.

To recap, we know these things about the twitter data:
- username mentions should have the tag 'USR'
- hashtags should have the tag 'HT'
- retweet tokens should have the tag 'RT'
- URL tokens should have the tag 'URL'

Remember how we replace these tokens with unique special ones (such as 'USER_TOKEN')? Your task is to adapt the emission probabilities for these tokens. Modify the emission matrix: assign 1.0 probability for the emission P('USER_TOKEN'|'USR') and 0.0 for P(word|'USR') for all other words. Do the same for the other three special tags.

In order to do that, you should use the vocabulary and tagset dictionaries in order to obtain the indices for the corresponding words and tags. Then, use the indices to find the values in the emission matrix and modify them. Print your new emission matrix. (0.5)

In [8]:
def adapt(emission):
    index_USR = tag_numbers['USR']
    index_HT = tag_numbers['HT']
    index_RT = tag_numbers['RT']
    index_URL = tag_numbers['URL']

    for i in range(len(word_names)):
        if word_names[i] == 'USER_TOKEN':
            emission[index_USR][i] = 1.0
        else:
            emission[index_USR][i] = 0.0
        
        if word_names[i] == 'HASHTAG_TOKEN':
            emission[index_HT][i] = 1.0
        else:
            emission[index_HT][i] = 0.0
        
        if word_names[i] == 'RETWEET_TOKEN':
            emission[index_RT][i] = 1.0
        else:
            emission[index_RT][i] = 0.0
        
        if word_names[i] == 'URL_TOKEN':
            emission[index_URL][i] = 1.0
        else:
            emission[index_URL][i] = 0.0
            
    return emission

emission = adapt(emission)


print(emission)

[[  9.15369893e-05   1.74752434e-04   8.32154448e-06 ...,   8.32154448e-06
    8.32154448e-06   8.32154448e-06]
 [  1.33457894e-05   1.33457894e-05   6.51955158e-01 ...,   1.33457894e-05
    1.33457894e-05   1.33457894e-05]
 [  1.62522347e-05   1.62522347e-05   1.62522347e-05 ...,   1.62522347e-05
    1.62522347e-05   1.62522347e-05]
 ..., 
 [  3.83582662e-05   3.83582662e-05   3.83582662e-05 ...,   3.83582662e-05
    3.83582662e-05   3.83582662e-05]
 [  3.83582662e-05   3.83582662e-05   3.83582662e-05 ...,   3.83582662e-05
    3.83582662e-05   3.83582662e-05]
 [  3.83582662e-05   3.83582662e-05   3.83582662e-05 ...,   3.83582662e-05
    3.83582662e-05   3.83582662e-05]]


<b>Instructions</b>: now evaluate your new tagger on the test tweets again. You should report accuracy but also do a fine-grained error analysis. Print the F-scores for **each tag**. <b>Hint:</b> use the "classification_report" function in scikit-learn for that. You should report the tags that performed the best and the worse. (0.5) 

In [9]:
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_recall_fscore_support

adapted_pred_tags = []
for tweet in word_index_test_tweets:
    adapted_pred_tags.append(viterbi((initial, transition, emission), tweet))

adapted_tags_of_predictions = []
length = len(adapted_pred_tags)
for i in range(length):
    adapted_tags_of_predictions += tag_sequence(adapted_pred_tags[i])

print('accuracy:')
print(accuracy_score(tags_of_test_tweets, adapted_tags_of_predictions))

_, _, f, _ = precision_recall_fscore_support(tags_of_test_tweets, adapted_tags_of_predictions, labels = tag_names)

worst_tags = []
best_tags = []
worst_score = 1
best_score = 0
for i in range(len(tag_names)):
    if f[i] > best_score:
        best_score = f[i]
        best_tags = [tag_names[i]]
    elif f[i] == best_score:
        best_score = f[i]
        best_tags.append(tag_names[i])
        
    if f[i] < worst_score:
        worst_score = f[i]
        worst_tags = [tag_names[i]]
    elif f[i] == worst_score:
        worst_score = f[i]
        worst_tags.append(tag_names[i])

print('best:')
print(best_tags)
print('worst:')
print(worst_tags)
print('\n\n')
print(classification_report(tags_of_test_tweets, adapted_tags_of_predictions))


accuracy:
0.695093842608
best:
['RT']
worst:
['-NONE-', '``', '$', 'NNPS', 'WP$', '-LRB-', 'PDT', 'FW', 'SYM', 'LS', '#', 'VPP', 'TD', 'O']



             precision    recall  f1-score   support

          #       0.00      0.00      0.00         0
          $       0.00      0.00      0.00         0
         ''       0.03      0.20      0.06        91
          ,       0.85      1.00      0.92       303
      -LRB-       0.00      0.00      0.00        32
     -NONE-       0.00      0.00      0.00         2
      -RRB-       0.04      0.15      0.07        34
          .       0.72      0.83      0.77       875
          :       0.97      0.76      0.85       562
         CC       0.96      0.88      0.92       305
         CD       0.59      0.59      0.59       268
         DT       0.74      0.93      0.82       825
         EX       0.38      0.80      0.52        10
         FW       0.00      0.00      0.00         3
         HT       0.98      0.98      0.98       135
        

C:\Users\msi\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\msi\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


<b>Instructions</b>: finally, based on the information you got above, do some analysis. Why do you think the tagger performed worse on the tags you mentioned above? How would you improve the tagger? Feel free to inspect some instances manually if you want (and show us if you do). Write your analysis in the markdown cell below. Notice that this question is inherently subjective: this is on purpose as you will be evaluated on your analytical abilities. But don't worry about going into depth: 2-4 sentences is enough (but feel free to write more if you need). (0.5)
    

<b>WRITE YOUR ANALYSIS HERE: </b>
The training set is Penn Treebank which is a small set, and many words or tags distribution patterns in tweets might not occur in PTB. Therefore, only using PTB as a training set to predict tweets is not good enough. A better way is to use a larger set of training data. Hence, we could use the PTB to build our initial, transition and emission matrices as a starting point, then we could apply unsuperivsed learning on a untagged tweets set. The results might get better.

### Extra credits 1: Expectation-Maximisation (1.0)

<b>Instructions</b>: here your goal is to improve the tagger using **hard EM**. This question is divided in two parts. Because EM can take a long time to run we will modify our code above to make it faster and also more robust to underflow by making calculations in the log space.

Your first task is to modify the `count` and `viterbi` functions. For `count`, you should return log probabilities for all matrices. For `viterbi`, you should modify the code in the following way:
- Calculate scores using log probabilities. Remember that in log space, any products become sums. Also remember to make sure you change the base case as well (not only the inner loop).
- You should rewrite the algorithm in vectorised form, in order to make it more efficient. Remember that in Viterbi, the third (inner) `for` loop calculate scores for a single cell, while the second `for` loop calculates scores for a whole column. These operations can be made in parallel, which makes them amenable to vectorisation. Replace the second and third `for` loops in the code with appropriate vector and matrix operations. Remember to do the same to obtain the backpointers. <b>Hint</b>: start by vectorising the inner loop only and check if it is correct.

The second task is to implement EM. This can be done without the first step above but bear in mind that it will take much longer to run. Write a function that:

- 1) Tag a corpus using Viterbi and an initial tagger
- 2) Train a new tagger using the tagged corpora obtained above.
- 3) Repeat both steps above N times.

If you've done the main homework correctly you should be able to reuse the `count` and `viterbi` functions for this and the code should be very straightforward. You should pretrain your tagger using the steps in the main homework, including the tag adaptation in Part 3. Then, in the inner loop, use the training tweets as your unlabelled corpora. Run 5 iterations of EM and report test accuracy **at every iteration**.

EM can take a long time to train, even if you're using the vectorised Viterbi code. If it is too slow in your machine, you're allowed to use a subset of the training tweets for this task.

To get full marks, adapt the algorithm in the following manner:
- At step 2) above, when training the new tagger, combine **both** the PTB gold data and the tagged training tweets, instead of just using the training tweets.

This is an easy trick to obtain better results (and it is essentially **semi-supervised** learning).

In [10]:
# load these functions used by both soft EM and hard EM
import numpy as np

def log_count(tagged_corpus, vocabulary, tagset):
    S = len(tagset)
    V = len(vocabulary)

    # initalise
    eps = 0.1
    pi = eps * np.ones(S)
    A = eps * np.ones((S, S))
    O = eps * np.ones((S, V))

    # count
    for sent in tagged_corpus:
        last_tag = None
        for word, tag in sent:
            O[tag, word] += 1
            if last_tag == None:
                pi[tag] += 1
            else:
                A[last_tag, tag] += 1
            last_tag = tag
        
    # normalise
    pi /= np.sum(pi)
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
        
    return np.log(pi), np.log(A), np.log(O)

def log_adapt(log_emission):
    index_USR = tag_numbers['USR']
    index_HT = tag_numbers['HT']
    index_RT = tag_numbers['RT']
    index_URL = tag_numbers['URL']

    for i in range(len(word_names)):
        if word_names[i] == 'USER_TOKEN':
            log_emission[index_USR][i] = 0
        else:
            log_emission[index_USR][i] = float('-inf')
        
        if word_names[i] == 'HASHTAG_TOKEN':
            log_emission[index_HT][i] = 0
        else:
            log_emission[index_HT][i] = float('-inf')
        
        if word_names[i] == 'RETWEET_TOKEN':
            log_emission[index_RT][i] = 0
        else:
            log_emission[index_RT][i] = float('-inf')
        
        if word_names[i] == 'URL_TOKEN':
            log_emission[index_URL][i] = 0
        else:
            log_emission[index_URL][i] = float('-inf')
            
    return log_emission

def log_viterbi(params, observations):
    pi, A, O = params
    M = len(observations)   # get number of input words
    S = pi.shape[0]         # get number of tags
    
    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf')
    backpointers = np.zeros((M, S), 'int')
    
    # base case
    alpha[0, :] = pi + O[:,observations[0]]      # transmition X emission
    
    for t in range(1, M):
        matrix_alpha = np.multiply(np.ones((S, 1)), alpha[t-1, :]).T
        matrix_O = np.multiply(np.ones((S, 1)), O[:, observations[t]].T)
        scores = matrix_alpha + A + matrix_O  # a S by S matrix
        
        score_max = np.max(scores, axis = 0)
        indices = np.argmax(scores, axis = 0)
        alpha[t, :] = score_max
        backpointers[t, :] = indices
    
    # now follow backpointers to resolve the state sequence
    reversed_pred_tags = []
    # return the last word index and the tag index of the largest score (word_index, tag_index)
    reversed_pred_tags.append((observations[-1], np.argmax(alpha[M-1,:])))
    for i in range(M-1, 0, -1):
        reversed_pred_tags.append((observations[i-1], backpointers[i, reversed_pred_tags[-1][1]]))
        
    return list(reversed(reversed_pred_tags))

In [11]:
# hard EM:

log_initial, log_transition, log_emission = log_count(num_corpus, word_numbers, tag_numbers)
# adapt emission
log_emission = log_adapt(log_emission)

length_training_set = len(encoded_tweets)
training_set = encoded_tweets

iteration = 5
for i in range(iteration):
    
    tagged_training_set = []
    for tweet in training_set:
        tagged_training_set.append(log_viterbi((log_initial, log_transition, log_emission), tweet))
    
    corpus =  num_corpus + tagged_training_set
    log_initial, log_transition, log_emission = log_count(corpus, word_numbers, tag_numbers)
    
    # adapt emission
    log_emission = log_adapt(log_emission)
    
    prediction = []
    original = []
    
    for sent in test_tweets:
        word_index = []
        for word, tag in sent:
            original.append(tag)
            word_index.append(word)
        prediction_sent = log_viterbi((log_initial, log_transition, log_emission), word_index)
        for word, tag in prediction_sent:
            prediction.append(tag)
    
    print(accuracy_score(original, prediction))

0.701020744155
0.70082318077
0.699637800461
0.695554823839
0.690549884755


### Extra credits 2: Soft EM using Forward-Backward (1.0)

<b>Instructions</b>: This is only for the truly intrepid: expect a substantial amount of work to get full marks in this. The goal is to perform **soft EM** using the Forward-Backward algorithm and expected counts. You will need to implement and adapt a set of functions for this task:
- The `count` method, which will still be used for pretraining, needs to also calculate **final** probabilities and return these as part of the output. These are never used in Viterbi but are essential for Forward-Backward.
- Implement the `forward` function. If you want to work in log space you should be careful because it requires summing probabilities. Hint: check the Scipy function `logsumexp` for that. Remember: the algorithm is very similar to Viterbi so feel free to use it as a starting point. The function should return the matrix with the alpha values and the marginal probability for the sentence.
- Implement the `backward` function. This should return a matrix with the beta values and the marginal. Remember the very useful sanity check: for the same sentence and tagger, the marginals returned from `forward` and `backward` should match.
- Implement the `expected_count` function, which is similar to `count` in the sense that it trains a tagger and output new parameters. However, it should have the alpha and beta matrices as inputs, as well as the marginals.

After implementing all of this, rerun EM using the soft approach and evaluate it in a similar way as done in Extra credits 1.


In [12]:
# load these 4 functions used by soft EM
from scipy.misc import logsumexp

def log_count_new(tagged_corpus, vocabulary, tagset):
    S = len(tagset)
    V = len(vocabulary)

    # initalise
    eps = 0.1
    pi = eps * np.ones(S)
    A = eps * np.ones((S, S))
    O = eps * np.ones((S, V))
    end = eps * np.ones(S)

    # count
    for sent in tagged_corpus:
        last_tag = None
        # tag = None
        for word, tag in sent:
            O[tag, word] += 1
            if last_tag == None:
                pi[tag] += 1
            else:
                A[last_tag, tag] += 1
            last_tag = tag
        end[tag] += 1
        
    # normalise
    pi /= np.sum(pi)
    end /= np.sum(end)
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
        
    return np.log(pi), np.log(A), np.log(O), np.log(end)

# forward algorithm (in log space):
def log_forward(params, observations):
    pi, A, O, end= params
    M = len(observations)   # get number of input words
    S = pi.shape[0]         # get number of tags
    
    alpha = np.zeros((M, S))
    alpha[:,:] = float(0.0)
    
    # base case
    alpha[0, :] = pi + O[:,observations[0]]      # transition X emission
    
    for t in range(1, M):
        matrix_alpha = np.multiply(np.ones((S, 1)), alpha[t-1, :]).T
        matrix_O = np.multiply(np.ones((S, 1)), O[:, observations[t]].T)
        scores = matrix_alpha + A + matrix_O  # a S by S matrix
        
        # sum prob, take off the log first then sum, finally get log score again
        prob_sum = logsumexp(scores, axis = 0)
        alpha[t, :] = prob_sum
        
    marginal_probability = logsumexp(alpha[-1, :] + end)     
    return alpha, marginal_probability


def log_backward(params, observations):
    pi, A, O, end= params
    M = len(observations)   # get number of input words
    S = pi.shape[0]   # get number of tags
    
    beta = np.zeros((M, S))
    beta[:,:] = float(0.0)
    
    # base case
    beta[-1, :] = end      # transmition X emission
    for t in range(M-2, -1, -1):
        matrix_beta = np.multiply(np.ones((S, 1)), beta[t+1, :])
        matrix_O = np.multiply(np.ones((S, 1)), O[:, observations[t+1]].T)
        scores = matrix_beta + A + matrix_O  # a S by S matrix
        
        # sum prob, take off the log first then sum, finally get log score again
        prob_sum = logsumexp(scores, axis = 1)
        beta[t, :] = prob_sum
    
    marginal_probability = logsumexp(beta[0, :] + pi + O[:, observations[0]])
    return beta, marginal_probability

def log_expected_count(tagged_corpus, vocabulary, tagset, prev_start, prev_transition, prev_emission, prev_end):
    S = len(tagset)
    V = len(vocabulary)
    
    eps = 0.1
    start = np.log(eps * np.ones(S))
    transition = np.log(eps * np.ones((S, S)))
    emission = np.log(eps * np.ones((S, V)))
    end = np.log(eps * np.ones(S))
            
    for sent in tagged_corpus:
        last_tag = None
        
        word_only = []
        for w in sent:
            word_only.append(w[0])

        beta, marginal = log_backward((prev_start, prev_transition, prev_emission, prev_end), word_only)
        alpha, marginal = log_forward((prev_start, prev_transition, prev_emission, prev_end), word_only)
        for i in range(len(sent)):
            word = sent[i][0]
            tag = sent[i][1]
            p_word_tag = alpha[i, tag] + beta[i, tag] - marginal
            emission[tag, word] = logsumexp([emission[tag, word], p_word_tag])
            
            if last_tag == None:
                start[tag] += 0   # log(1) = 0
            else:
                epsilon = alpha[i, last_tag] + prev_transition[last_tag, tag] + prev_emission[tag, i] + beta[i, tag] - marginal
                transition[last_tag, tag] = logsumexp([transition[last_tag, tag], epsilon])
            last_tag = tag
        end[tag] += 0    # log(1) = 0
    
    # normalise
    start -= logsumexp(start)
    end -= logsumexp(end)
    for s in range(S):
        emission[s,:] -= logsumexp(emission[s,:])
        transition[s,:] -= logsumexp(transition[s,:])
            
    return prev_start, transition, emission, prev_end

In [13]:
# soft EM: this block of code take very long time to run (about 15 mins)

a, b, c, d = log_count_new(num_corpus, word_numbers, tag_numbers)
a, b, c, d = log_expected_count(num_corpus, word_numbers, tag_numbers, a, b, c, d)
# adapt emission
log_emission = log_adapt(log_emission)

length_training_set = len(encoded_tweets)
training_set = encoded_tweets

interation = 5
for i in range(interation):
    
    tagged_training_set = []
    for tweet in training_set:
        tagged_training_set.append(log_viterbi((a, b, c), tweet))
        
    corpus =  num_corpus + tagged_training_set
    a, b, c, d = log_count_new(corpus, word_numbers, tag_numbers)
    a, b, c, d = log_expected_count(corpus, word_numbers, tag_numbers, a, b, c, d)
    
    # adapt emission
    log_emission = log_adapt(log_emission)
    
    
    prediction = []
    original = []
    
    for sent in test_tweets:
        word_index = []
        for word, tag in sent:
            original.append(tag)
            word_index.append(word)
        prediction_sent = log_viterbi((a,b,c), word_index)
        for word, tag in prediction_sent:
            prediction.append(tag)
    
    print(accuracy_score(original, prediction))
    

0.624234441883
0.61969048403
0.61646361541
0.613368455713
0.612644056635
